# Convolution Neural Network avec Keras

In [2]:
import os
import cv2 
# si nécessaire : conda install -c conda-forge opencv   
# ou sinon : https://pypi.org/project/opencv-python/
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

On importe les librairies nécessaires de Keras

In [3]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adam, Adadelta
from keras.utils.np_utils import to_categorical

## Importation et préparation des données

Télécharger les données depuis :

https://box.ec-nantes.fr:443/index.php/s/2NxopNZS3FTRBcr

Adresse du dossier où sont entreposées les données:

In [4]:
#data_path = "data_animals"
data_path = "/Users/dgalembeck/Documents/Coding/Cours/STASC/Data/data_animals"

La fonction `os.listdir()` permet de lister le contenu du dossier `data_animals` (un repertoire par classe).

In [5]:
data_dir_list = os.listdir(data_path)
print(data_dir_list)
num_classes = len(data_dir_list) 
print(num_classes)

['dogs', 'cats', 'Humans', 'horses']
4


Toutes les images ne sont pas au même format (nb de pixels).
Le réseau CNN impose que toutes les données aient la même dimension. Il nous faudra  transformer les images pour qu'elles soient toutes au même format : 128x 128.

In [6]:
img_rows=128
img_cols=128

Lorsque les images en entrée du réseau sont en couleur, on utilise 3 canaux(RGB).

Ici, pour simplifier, nous allons préalablement transformer les images en niveaux de gris et de ce fait nous n'utiliserons qu'un seul canal en entrée du réseau. 

In [7]:
num_channel=1

> Compléter le script ci-dessous pour importer les images en niveaux de gris et sous la forme de tableaux 128x128, dans la liste img_data_list.
*  `my_img = cv2.imread("file")`  : lecture d'un fichier image
*  `cv2.cvtColor(my_img, cv2.COLOR_BGR2GRAY)` : convertit le fichier image en niveaux de gris
*  `cv2.resize(input_img,(n,p))` : redimensionne l'image au format n x p 

In [18]:
from sklearn import preprocessing

img_data_list=[]

for dataset in data_dir_list: # boucle sur les 4 repertoires
    img_list=os.listdir(data_path+'/'+ dataset)  # 
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        print(img)
        input_img_raw = cv2.imread("/Users/dgalembeck/Documents/Coding/Cours/STASC/Data/data_animals/" + dataset + '/' + img) 
        input_img_grey= cv2.cvtColor(input_img_raw, cv2.COLOR_BGR2GRAY)
        input_img_flatten=cv2.resize(input_img_grey,(128,128)).flatten()
        img_data_list.append(input_img_flatten)

        
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')

img_data_scaled = preprocessing.scale(img_data)
print (img_data_scaled.shape)

print (np.mean(img_data_scaled))
print (np.std(img_data_scaled))

print (img_data_scaled.mean(axis=0))
print (img_data_scaled.std(axis=0))  

Loaded the images of dataset-dogs

dog.167.jpg
dog.173.jpg
dog.36.jpg
dog.22.jpg
dog.198.jpg
dog.199.jpg
dog.23.jpg
dog.37.jpg
dog.172.jpg
dog.166.jpg
dog.170.jpg
dog.164.jpg
dog.21.jpg
dog.158.jpg
dog.35.jpg
dog.159.jpg
dog.34.jpg
dog.20.jpg
dog.165.jpg
dog.171.jpg
dog.149.jpg
dog.24.jpg
dog.30.jpg
dog.175.jpg
dog.18.jpg
dog.161.jpg
dog.160.jpg
dog.174.jpg
dog.19.jpg
dog.31.jpg
dog.148.jpg
dog.25.jpg
dog.202.jpg
dog.200.jpg
dog.33.jpg
dog.27.jpg
dog.162.jpg
dog.176.jpg
dog.189.jpg
dog.188.jpg
dog.177.jpg
dog.163.jpg
dog.26.jpg
dog.32.jpg
dog.201.jpg
dog.104.jpg
dog.69.jpg
dog.110.jpg
dog.138.jpg
dog.55.jpg
dog.1.jpg
dog.41.jpg
dog.96.jpg
dog.82.jpg
dog.83.jpg
dog.97.jpg
dog.40.jpg
dog.139.jpg
dog.54.jpg
dog.111.jpg
dog.105.jpg
dog.68.jpg
dog.113.jpg
dog.107.jpg
dog.2.jpg
dog.42.jpg
dog.56.jpg
dog.81.jpg
dog.95.jpg
dog.94.jpg
dog.80.jpg
dog.57.jpg
dog.43.jpg
dog.3.jpg
dog.106.jpg
dog.112.jpg
dog.47.jpg
dog.7.jpg
dog.53.jpg
dog.116.jpg
dog.102.jpg
dog.84.jpg
dog.90.jpg
dog.91.jpg
dog.85

/Users/dgalembeck/miniforge3/envs/IA/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/dgalembeck/miniforge3/envs/IA/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


> Quelle la dimension du tableau `img_data` ?

In [22]:
img_data.shape # 808 images with 128*128 features each (pixels in grayscale)

(808, 16384)

De façon générale, la première couche du réseau de convolution  prend en entrée un objet de dimension 3 : hauteur, largeur, profondeur,  où la profondeur correspond aux nombres de canaux.

Avec Tensor Flow (ici en backend) la profondeur doit être donnée en dernière position.

Cette dimension est ici "factice"  car nos images sont en niveaux gris, elle est néanmoins nécessaire car attendue par les fonctions de Keras et Tensor Flow.

L'échantillon d'images doit finalement se présenter sous la forme d'un objet de dimension 4: (nombre d'échantillons, hauteur, largeur, profondeur)

Nous redimensionnons les données pour qu'elle se présente ainsi :

In [23]:
img_data_reshape=img_data_scaled.reshape(img_data.shape[0],
                                        img_rows,img_cols,
                                        num_channel)
print (img_data_reshape.shape)

(808, 128, 128, 1)


La dimension d'une image en entrée du réseau est la suivante :

In [24]:
input_shape=img_data_reshape[0].shape # (128, 128, 1)
input_shape

(128, 128, 1)

Nous indiquons maintenant les labels des images:

In [25]:
num_of_samples = img_data_reshape.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')
labels[0:202]=0
labels[202:404]=1
labels[404:606]=2
labels[606:]=3

names = ['cats','dogs','horses','humans']

> Convertir les labels en "one-hot encoding"

In [26]:
labels = to_categorical(labels)
labels

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

> Séparer  aléatoirement les données en un échantillon d'apprentissage (80%) et un échantillon de test (20%). Assurez-vous que les données d'apprentissage prennent bien la forme d'un tableau de dimension 4.

In [27]:
X_train, X_test, y_train, y_test = train_test_split(img_data, labels, test_size=0.2)

## Définition de l'architecture du modèle

Nous définissons ci-dessous les deux premiers niveaux de convolution du réseau CNN.
Chacune de ces deux couches est définie comme suit :
+ 32 noyaux (filtres)
+ Pas (stride) = 1
+ Kernel size = (3,3)
+ padding = 'same' (i.e. 0 padding : bordures à 0)
+ activation : relu

> Créer un modèle séquentiel que vous nommerez `my_first_CNN` composé de 4 couches succesives (conv + relu + conv + relu).    
> Voir  [ici](https://keras.io/layers/convolutional/#conv2d) et  [ici](https://keras.io/examples/vision/mnist_convnet/) pour la synthaxe de la couche de convolution `Conv2D`.

In [28]:
from tensorflow import keras
from tensorflow.keras import layers

my_first_CNN = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        #layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        #layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        #layers.Dropout(0.5),
        #layers.Dense(num_classes, activation="softmax"),
    ]
)

my_first_CNN.summary()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-12-16 11:55:32.399689: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-16 11:55:32.401267: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 57600)             0         
                                                                 
 dropout (Dropout)           (None, 57600)             0

> Executez les codes ci-dessous et decrire les sorties obtenues

In [ ]:
print(my_first_CNN.layers[0].input_shape)
print(my_first_CNN.layers[1].input_shape)

(`batch_size`,`n_l`,`n_c`,`nb de canaux`)

Keras dimensionnera ensuite correctement les couches en fonction du `batch_size` choisi par l'utilisateur. 

> Passer `batch_size=16` en argument de `Conv2D` et vérifier que cela a bien été pris en compte dans les dimensions de la couche cachée de `my_first_CNN`.

> Executez les codes ci-dessous et decrire les sorties obtenues. Expliquer en particulier la dimension de la troisième couche en utilisant `get_weights`.

In [ ]:
my_first_CNN.summary()

In [ ]:
np.shape(my_first_CNN.layers[2].get_weights()[0])

Multichannel convolution:
\begin{eqnarray} Z(i,j,l) &= & ( V \star K ) (i,j,l) \\
& =  & \sum_{u,v,w}   V (i+u, j+v,w ) w_{u,v,l,w} 
\end{eqnarray}
where 
+ $V$  and $Z$ have the same dimensions (multichannel).
+ $K(u,v,l,w) $ gives the connection strength between a unit in channel $l$ of the output and a unit in channel $w$ of the input, with an offset of  $u$ rows and $v$ columns between the output unit and the input unit.

Le nombre de poids à estimer vaut donc :

Le nombre de paramètres à estimer pour les termes de biais :

On a bien que pour la troisième couche 9248 = 

> Vérifier que les poids sont (déjà) initialisés aléatoirement alors que les biais sont initialisés à 0. 

> Construire maintenant l'architecture complète du réseau `my_first_CNN` :
+ Convolution à 32 filtres de taille (3,3), zero padding
+ Activation Relu
+ Convolution à 32 filtres de taille (3,3), zero padding
+ Activation Relu
+ Maxpooling2D (2,2) [documentation](https://keras.io/api/layers/pooling_layers/max_pooling2d/)
+ Dropout(0.5) [documentation](https://keras.io/api/layers/regularization_layers/dropout/)
+ Convolution à 64 filtres de taille (3,3), zero padding
+ Maxpooling2D (2,2) 
+ Dropout(0.5) 
+ Flatten  [documentation](https://keras.io/api/layers/reshaping_layers/flatten/)
+ Dense(64)
+ Activation Relu
+ Dropout(0.5)
+ Dense(4)
+ Softmax   
> 
> Afficher un résumé de l'architecture avec `my_first_CNN.summary` 

## Apprentissage du CNN

> Ajuster le modèle 
- avec la méthode sgd (avec un taux d'apprentissage de 0.01 et momentum de 0.9)
- puis la méthode adam.

> Tracer en fonction du nombre d'epochs le risque de cross-entropy ainsi que la précision pour les échantillons d'apprentissage et de validation.

> Donner le risque de cross-entropy ainsi que la précision pour l'échantillon de test.

## Matrice de confusion

> Utilisez les outils `classification_report()` et `confusion_matrix()` de `sklearn.metrics` pour décrire les performances du réseau de neurones.

Pour afficher la matrice de confusion sous forme graphique, on dispose de la fonction [`plot_confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html) de sklearn. Pour pouvoir utiliser les fonctionnalités de sklearn, il nous faut tout d'abord transformer l'objet Keras en un classifieur sklearn. On peut pour cela utiliser la fonction wrapper [`KerasClassifier`](https://www.tensorflow.org/api_docs/python/tf/keras/wrappers/scikit_learn/KerasClassifier) du module `keras.wrappers.scikit_learn` (voir aussi le TP précédent).

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

La fonction `KerasClassifier` est le plus souvent utilisée pour ajuster un réseau, typiquement pour une procédure de type validation croisée (`Gridsearch`). Ici, au contraire, on ne souhaite pas réajuster une nouvelle fois le modèle, mais uniquement changer sa forme. 

> Compléter le code ci-dessous pour créer l'objet `wrapped_model`

In [ ]:
wrapped_model = KerasClassifier(build_fn = lambda : ### TO DO ###,
                                epochs = ### TO DO ###) 

> Ajuster ce modèle sur les données d'apprentissage. Assurez vous que les prédictions de `wrapped_model` et de `my_first_CNN` sur les données de test sont bien identiques.

> Essayer maintenant d'appliquer la fonction `plot_confusion_matrix` au modèle `wrapped_model`et aux données de test.

Cela ne fonctionne pas, il y a en effet un petit bug dans la fonction `KerasClassifier`. En étudiant l'erreur renvoyée ci-dessus, on comprend que la fonction `plot_confusion_matrix` teste si `wrapped_model` est un classifieur, et que le test ici ne passe pas :

In [ ]:
from sklearn.base import is_classifier
is_classifier(wrapped_model)

Le problème vient du fait que wrapped_model ne possède pas d'attribut "_estimator_type" :

In [ ]:
wrapped_model._estimator_type

> Utiliser la fonction [`setattr`](https://docs.python.org/3/library/functions.html#setattr) pour résoudre ce problème et applique finalement la fonction `plot_confusion_matrix`.

## Sauvegarde d'un réseau de neurones avec Keras

Lorsqu'un modèle a été ajusté, on peut vouloir conserver 
- l'architecture du réseau
- la valeurs des poids des couches
- l'optimiseur utilisé pour ajuster les poids 
- les métriques et les pertes considérées 

Pour répondre aux questions ci-dessous, vous pourrez consulter cette [page](https://keras.io/guides/serialization_and_saving/) de la documentation qui présente en détail les méthodes pour sauvegarder des réseaux Keras.

##### Sauvegarde du modèle (architecture seule) en json

Dans certaines situations, on ne souhaite sauvegarder que l'architecture. Par exemple si on veut comparer plusieurs méthodes d'optimisation des poids d'un même réseau. Il est possible de sauvegarder l'architecture d'un réseau au format JSON.

JavaScript Object Notation (JSON) est un format de données textuelles dérivé de la notation des objets du langage JavaScript. Il permet de représenter de l’information structurée.

Un document JSON a pour fonction de représenter de l'information accompagnée d'étiquettes permettant d'en interpréter les divers éléments, sans aucune restriction sur le nombre de celles-ci.

Un document JSON ne comprend que deux types d'éléments structurels :
+ Des ensembles de paires "nom" (alias "clé") / "valeur" ;
+ Des listes ordonnées de valeurs.

> Sauver l'architecture du réseau au format json.   
> Afficher le contenu du fichier sauvé.   
> Quelle est la taille du fichier json sur votre disque ?

##### Sauvegarde et chargement du modèle complet entrainé

> Utiliser les fonctions `model.save()` et `load_model()` pour sauver et charger un modèle complet (architecture, poids, optimiseur, métriques). Quelle est la taille du répertoire créé pour cette sauvegarde ?

> Comparer les poids du réseau reconstruit aux poids du réseau originel.

> Vérifier que le modèle chargé peut être directement utilisé pour faire des prédictions ou pour calculer un score.

## Bonus : ajustement du modèle sur Google Colab

> Ajuster ce modèle CNN (ou évenuellement un modèle plus profond) sur [Google Colab](https://colab.research.google.com/notebooks) (ou sur [Binder](https://mybinder.org/)). 
> Il vous faudra telecharger les données sur Colab et adapter les codes du TP pour l'importation des images, plusieurs solutions sont possibles, voir par exemple
[ici](https://towardsdatascience.com/importing-data-to-google-colab-the-clean-way-5ceef9e9e3c8]).